In [87]:
!pip install quandl

In [88]:
#Install Dependencies 
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [89]:
#Get Stock Data
df = quandl.get("WIKI/FB")
# Print data
print(df.head())

             Open   High    Low    Close       Volume  Ex-Dividend  \
Date                                                                 
2012-05-18  42.05  45.00  38.00  38.2318  573576400.0          0.0   
2012-05-21  36.53  36.66  33.00  34.0300  168192700.0          0.0   
2012-05-22  32.61  33.59  30.94  31.0000  101786600.0          0.0   
2012-05-23  31.37  32.50  31.36  32.0000   73600000.0          0.0   
2012-05-24  32.95  33.21  31.77  33.0300   50237200.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2012-05-18          1.0      42.05      45.00     38.00     38.2318   
2012-05-21          1.0      36.53      36.66     33.00     34.0300   
2012-05-22          1.0      32.61      33.59     30.94     31.0000   
2012-05-23          1.0      31.37      32.50     31.36     32.0000   
2012-05-24          1.0      32.95      33.21     31.77     33.0300   

           

In [90]:
# Get Adj Close Price
df = df[['Adj. Close']]
print(df.head())

            Adj. Close
Date                  
2012-05-18     38.2318
2012-05-21     34.0300
2012-05-22     31.0000
2012-05-23     32.0000
2012-05-24     33.0300


In [91]:
#Predict 'forecast_out' days out in the future
forecast_out = 2
# Create another column (Dependent Variable) and shifted 'forecast_out' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
print(df.tail())


            Adj. Close  Prediction
Date                              
2018-03-21      169.39      159.39
2018-03-22      164.89      160.06
2018-03-23      159.39      152.19
2018-03-26      160.06         NaN
2018-03-27      152.19         NaN


In [92]:
# Create an independent data set (X)
# Convert df to a numpy array
X = np.array(df.drop(['Prediction'],1))
#Remove the last 'forecast_out' rows
X = X[:-forecast_out]
print(X)


[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [169.39  ]
 [164.89  ]
 [159.39  ]]


In [93]:
# Create the dependent data set(Y)
#Convert df to a numpy array(All of values including the NaNs)
Y = np.array(df['Prediction'])
#Get all of the Y values except the last 'forecast_out' rows
Y = Y[:-forecast_out]
print(Y)

[ 31.    32.    33.03 ... 159.39 160.06 152.19]


In [94]:
# Split Data into 80% training and 20% test
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

In [95]:
# Create and train the Support Vector Machine (Regression Model)
svr_rbf = SVR(kernel = 'rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [96]:
# Testing Model: Score returns the coefficent of determination R^2 of the prediction.
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test,y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9975767731994788


In [97]:
# Create and train the Linear Regression Model
linear_regression = LinearRegression()
# Train Model
linear_regression.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [98]:
# Test the Linear Regression Model
# Testing Model: Score returns the coefficent of determination R^2 of the prediction.
# The best possible score is 1.0
linear_regression_confidence = linear_regression.score(x_test,y_test)
print("lr confidence: ", linear_regression_confidence)

lr confidence:  0.9982255247481733


In [99]:
# Set x_forecast = to last 30 rows of the orginal data set (Adj. close)
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[160.06]
 [152.19]]


In [100]:
# Print linear_regression model predictions
linear_regression_prediction = linear_regression.predict(x_forecast)
print("Linear Regression Model:")
print(linear_regression_prediction)

print() #gap

print("SVM Regressor Model:")
#Print svm regressor model predictions
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

Linear Regression Model:
[160.17004993 152.30777958]

SVM Regressor Model:
[163.09813574 151.63924113]
